In [112]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

#df = pd.read_csv('./data/boston_airbnb/boston_listings.csv')
df_listings = pd.read_csv('./data/boston_airbnb/listings.csv')
df_listings.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,12147973,https://www.airbnb.com/rooms/12147973,20160906204935,2016-09-07,Sunny Bungalow in the City,"Cozy, sunny, family home. Master bedroom high...",The house has an open and cozy feel at the sam...,"Cozy, sunny, family home. Master bedroom high...",none,"Roslindale is quiet, convenient and friendly. ...",...,NaN,f,NaN,NaN,f,moderate,f,f,1,NaN
1,3075044,https://www.airbnb.com/rooms/3075044,20160906204935,2016-09-07,Charming room in pet friendly apt,Charming and quiet room in a second floor 1910...,Small but cozy and quite room with a full size...,Charming and quiet room in a second floor 1910...,none,"The room is in Roslindale, a diverse and prima...",...,9.0,f,NaN,NaN,t,moderate,f,f,1,1.30
2,6976,https://www.airbnb.com/rooms/6976,20160906204935,2016-09-07,Mexican Folk Art Haven in Boston,"Come stay with a friendly, middle-aged guy in ...","Come stay with a friendly, middle-aged guy in ...","Come stay with a friendly, middle-aged guy in ...",none,The LOCATION: Roslindale is a safe and diverse...,...,10.0,f,NaN,NaN,f,moderate,t,f,1,0.47
3,1436513,https://www.airbnb.com/rooms/1436513,20160906204935,2016-09-07,Spacious Sunny Bedroom Suite in Historic Home,Come experience the comforts of home away from...,Most places you find in Boston are small howev...,Come experience the comforts of home away from...,none,Roslindale is a lovely little neighborhood loc...,...,10.0,f,NaN,NaN,f,moderate,f,f,1,1.00
4,7651065,https://www.airbnb.com/rooms/7651065,20160906204935,2016-09-07,Come Home to Boston,"My comfy, clean and relaxing home is one block...","Clean, attractive, private room, one block fro...","My comfy, clean and relaxing home is one block...",none,"I love the proximity to downtown, the neighbor...",...,10.0,f,NaN,NaN,f,flexible,f,f,1,2.25


In [28]:
df_listings.shape

(3585, 94)

In [71]:
#columns with no nulls
missing_cols = set(df_listings.columns[df_listings.isnull().mean()==0.0])
print(missing_cols)


{'state', 'requires_license', 'host_url', 'id', 'room_type', 'extra_people', 'host_listings_count', 'is_location_exact', 'host_total_listings_count', 'host_has_profile_pic', 'picture_url', 'host_name', 'accommodates', 'country', 'host_is_superhost', 'latitude', 'longitude', 'smart_location', 'availability_365', 'experiences_offered', 'host_since', 'neighbourhood_cleansed', 'calendar_updated', 'last_scraped', 'host_id', 'maximum_nights', 'availability_60', 'require_guest_profile_picture', 'country_code', 'amenities', 'host_thumbnail_url', 'price', 'calculated_host_listings_count', 'cancellation_policy', 'host_identity_verified', 'name', 'instant_bookable', 'availability_30', 'require_guest_phone_verification', 'host_picture_url', 'availability_90', 'host_verifications', 'listing_url', 'calendar_last_scraped', 'description', 'bed_type', 'minimum_nights', 'number_of_reviews', 'scrape_id', 'street', 'guests_included'}


In [68]:
#columns with more than 50% missing values
missing_cols = set(df_listings.columns[df_listings.isnull().mean()>0.5])
print(missing_cols)

{'license', 'square_feet', 'weekly_price', 'notes', 'neighbourhood_group_cleansed', 'security_deposit', 'monthly_price', 'has_availability', 'jurisdiction_names'}


In [143]:
#fractions in each of the neighbourhoods (highest 20)
vals = df_listings['neighbourhood'].value_counts()
#print(vals)
((vals[:20])/df_listings.shape[0]).plot(kind="bar");

Allston-Brighton      364
Jamaica Plain         314
South End             298
Back Bay              291
Fenway/Kenmore        249
South Boston          216
Dorchester            195
Beacon Hill           174
North End             125
East Boston           117
Roxbury               116
Mission Hill          103
Charlestown            79
Chinatown              78
West End               68
Roslindale             50
West Roxbury           35
Theater District       33
Downtown Crossing      26
Hyde Park              25
Mattapan               20
Financial District     13
Somerville             13
Leather District        8
Downtown                8
Brookline               8
Cambridge               7
Chestnut Hill           4
Government Center       3
Harvard Square          2
Name: neighbourhood, dtype: int64
['Roslindale' nan 'Jamaica Plain' 'Mission Hill' 'Fenway/Kenmore'
 'Back Bay' 'Leather District' 'Chinatown' 'Hyde Park' 'North End'
 'Roxbury' 'South End' 'Mattapan' 'East Boston' 'Sout

In [77]:
df_scoreloc = df_listings[['name','review_scores_location']]
df_scoreloc.head()

,name,review_scores_location
0,Sunny Bungalow in the City,NaN
1,Charming room in pet friendly apt,9.0
2,Mexican Folk Art Haven in Boston,9.0
3,Spacious Sunny Bedroom Suite in Historic Home,10.0
4,Come Home to Boston,9.0


In [119]:
#remove $ signs from price
#explore relationship between price and cancellation policy for single neighbourhood

df = df_listings.copy()
df.price = df.price.apply(lambda x: x.strip('$'))
df.price =  pd.to_numeric(df['price'],errors='coerce')
#df.price.head()

pd.DataFrame(df.query("neighbourhood == 'Allston-Brighton'").groupby(['cancellation_policy']).mean()['price']  )

,price
cancellation_policy,
flexible,99.302326
moderate,118.616438
strict,123.987654


In [120]:
#remove $ signs from price - create new column
df_listings['price_val'] = df_listings.price.apply(lambda x: x.strip('$'))
df_listings['price_val']  =  pd.to_numeric(df_listings['price_val'] ,errors='coerce')
df_listings['price_val'].head()

0    250.0
1     65.0
2     65.0
3     75.0
4     79.0
Name: price_val, dtype: float64

In [121]:
#mean price
df_listings['price_val'].mean()

169.07276798208787

In [129]:
def room_private(room_type_str):
    '''
    INPUT
        room_type_str - a string to specify the room type
    
    OUTPUT
        return 1 if the string is in ("Private room")
        return 0 otherwise
    '''
    if room_type_str in ("Private room"):
        return 1
    else:
        return 0



In [131]:
df_listings['pvt_room'] = df_listings["room_type"].apply(room_private)
pvt_room_perc = df_listings['pvt_room'].mean()
print(pvt_room_perc)

0.38437935843793586


In [132]:
df_pvt = df_listings[df_listings['pvt_room']==1] 
df_not_pvt = df_listings[df_listings['pvt_room']==0] 

print(df_t[['name','price_val']][:5])

print(df_t['price_val'].mean())
print(df_f['price_val'].mean())

                                            name  price_val
0                     Sunny Bungalow in the City      250.0
1              Charming room in pet friendly apt       65.0
2               Mexican Folk Art Haven in Boston       65.0
3  Spacious Sunny Bedroom Suite in Historic Home       75.0
4                            Come Home to Boston       79.0
169.06816269284712
171.125


In [145]:
df_listings.groupby(['bed_type']).mean()['price_val'].sort_values().dropna()


bed_type
Airbed            78.675000
Futon             85.254902
Couch             99.600000
Pull-out Sofa    101.612903
Real Bed         172.175530
Name: price_val, dtype: float64